In [ ]:
%matplotlib inline
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
from shapely.geometry import mapping
sys.path.append(r'E:/gitlab/geoloc2/Detecciondeterrenos')
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r'C:/Users/ASUS/Inteligencia_Artificial/yolov7')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import time
import datetime

In [ ]:
def matchingTemplate(img_rgb,template,nameimg,threshold = .7):
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    w, h = template.shape[::-1]
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    loc = np.where( res >= threshold)
    startpoints=[]
    endpoints=[]
    for pt in zip(*loc[::-1]):
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
        startpoints.append(pt)
        endpoints.append((pt[0] + w, pt[1] + h))
    return img_rgb,startpoints,endpoints

In [ ]:
raster=r"C:\Users\ASUS\Desktop\capa_naucalpan\capa_naucalpan.tif"
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy=Parametro_raster(raster,metros=1000)

In [ ]:
chincheta=[]
clase=[]
template = cv2.imread(r"C:\Users\ASUS\Desktop\presentacionyolov7\etiquetas_google1\etiquetas_google1\poli.png")
template_gray=template[:,:,0]
with rasterio.open(raster) as src:
    with tqdm.tqdm(total=alto*ancho*4) as pbar:
        for j in range(ancho*2):#ancho
            for i in range(alto*2):#alto
                generar=0
                label=raster.replace('\\','/').split('/')[-1][:-4]+'_'
                nameimg=label.lower()+str(i)+'_'+str(j)
                cuadro=[]
                for k in range(2):
                    for l in range(2):
                        cuadro.append((minx+(maxx-minx)/ancho*(j/2+k),maxy-(maxy-miny)/alto*(i/2+l)))
                cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
                shapes=[{'type':'Polygon','coordinates':[cuadro]}]
                vector=[]
                array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
                if np.sum(array)<100:
                    pbar.update(1)
                    continue
                four_images=[array[2],array[1],array[0]]
                imagen_n = np.stack(four_images, axis=-1)
                try:
                    imagen_n,startpoints, endpoints=matchingTemplate(imagen_n,template_gray,nameimg,threshold = .7)
                except:
                    pbar.update(1)
                    continue
                
                for i in range(len(startpoints)):
                    startpoint=startpoints[i]
                    endpoint=endpoints[i]
                    df_aux=imagen_n.copy()
                    x1,y1=startpoint
                    x2,y2=endpoint
                    df_aux=df_aux[y1:y2,x1:x2]    

                    startpoint=[x/dim for x in startpoint]
                    endpoint=[x/dim for x in endpoint]
                    proyecciones=shapes[0].get('coordinates')[0][:-1]
                    chincheta.append(rotacion_detect(startpoint,endpoint,0,proyecciones,dim,dim,dim))
#                     cv2.rectangle(imagen_n,startpoint,endpoint,(250,0,0),2)
#                 cv2.imshow("1",imagen_n)
#                 cv2.waitKey()
#                 cv2.destroyAllWindows()

                
                
                pbar.update(1)
                
                

In [ ]:
gps=gpd.GeoDataFrame(geometry=chincheta,crs=crs)
# gps["centroidx"]=gps.centroid.x/10
# gps["centroidy"]=gps.centroid.y/10
# gps["centroidx"]=gps["centroidx"].astype("int")
# gps["centroidy"]=gps["centroidy"].astype("int")
# gps=gps.drop_duplicates(["centroidx","centroidy"])
gps.to_file(r"C:\Users\ASUS\Desktop\capa_naucalpan\chinchetas/hoteles.shp")